# Data Analysis of "Value for money" flats

Name: Bryan Lim Heng Yi

Admin Number: P2204314

Diploma: Diploma in Computer Engineering

Class: DCPE/FT/2B/21

Module Class: EL/03


# Imports (Libraries)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mc
from tabulate import tabulate

# Data Sets


In [ ]:
# genfromtext used to read csv file

# Resale Flat Prices from year 2017 onwards [https://data.gov.sg/dataset/resale-flat-prices]
resale_price = "resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv"
data_sale = np.genfromtxt(resale_price,dtype=['datetime64[D]','U15','U9','U4','U20','U20',int,'U20',int,'U50',int],
                      delimiter=',', names=True, missing_values=['na'],filling_values=0)
print(tabulate(data_sale[:5],headers=data_sale.dtype.names),"\n")

# Median Rent by Town and Flat Type [https://data.gov.sg/dataset/median-rent-by-town-and-flat-type]
rental_price = "median-rent-by-town-and-flat-type.csv"
data_rent = np.genfromtxt(rental_price,dtype=['U7', 'U20', 'U5', int],
                      delimiter=',',names=True, missing_values=['na', '-'], filling_values=0)
print(tabulate(data_rent[:5],headers=data_rent.dtype.names),"\n")

# HDB Property Information [https://data.gov.sg/dataset/hdb-property-information]
info = "hdb-property-information.csv"
data_info = np.genfromtxt(info,dtype=['U4','U50',int,int,'U1','U1','U1','U1','U1','U1','U3',int,int,int,int,int,int,int,int,int,int,int,int,int],
                          delimiter=',',names=True, missing_values=['na'],filling_values=0) 
print(tabulate(data_info[:5],headers=data_info.dtype.names),"\n")

# Analysis: Dataset 1 [Resale Price]

This data set shows the different resale prices of flats over 7 years (2017 - 2023)

The flats are classified by their town, flat type, storey range, floor area, flat model, lease commence date and remaining lease



In [ ]:
print(f"There are {len(data_sale)} rows and {len(data_sale[0])} columns in this dataset.\n")
print(f"The names of the data are {data_sale.dtype.names}\n")
print(f"This dataset spans {len(np.unique(data_sale['month']))} months over 7 years.\n")

# Analysis: Dataset 2 [Rental Price]

In [ ]:
print(f"There are {len(data_rent)} rows and {len(data_rent[0])} columns in this dataset.\n")
print(f"The names of the data are {data_rent.dtype.names}\n")
print(f"This dataset spans {len(np.unique(data_rent['quarter']))} quarters over 18 years.\n")

# Analysis: Dataset 3 [Property Information]

This data set shows the different amounts of amenities around different blocks of flats

This data is going to be used to compare the correlation between flat prices against the amount of amenities in the area.

This data set is not very useful in its current state as there are different rows for different amenities, making the data hard to work with when we want to compare amenities in general.

In addition to that, the town names are in abbreviated form, making it hard to compare with the other dataset which town names are in full. The town 'Tengah' ('TG') is also missing in the dataset.

Further more, the 4 and 5 room rental flats are classified as 'other_room_rental', meaning that we can do not know how many of each type is being rented. 

Hence I had to clean the data to colate all of the different amenities into 1 data as well as to change the town names from abbreviations to full forms.

Since there are different number of data to compare, we have to take an average count of amentities for each town/area.

This data set would be used in Graph _.

In [ ]:
print(f"There are {len(data_info)} rows and {len(data_info[0])} columns in this dataset.\n")
print(f"The names of the data are {data_info.dtype.names}\n")
print(f"This data set covers {len(np.unique(data_info['bldg_contract_town']))} different areas of Singapore. \n")

# Cleaning Data set 2

### For Line Graph:
| **Problem** | **Solution** |
| ----------- | ----------- |
| Quarters need to be converted to years so that the number of rentals in that year and the price and be counted. | Iterate through the data set and finds each data with the corresponding year, storing the rental price and the the number of each year in lists.  |
| Needs to find average rent per year. | Took the 2 lists and calculated the average of each year.  |


In [ ]:
# Convert quarters to years
year_total = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
count_year = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
avg_rent = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
years = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

for i in range(len(data_rent)):
    if "2005" in data_rent['quarter'][i]:
        year_total[0] += data_rent['median_rent'][i]
        count_year[0] += 1
    elif "2006" in data_rent['quarter'][i]:
        year_total[1] += data_rent['median_rent'][i]
        count_year[1] += 1
    elif "2007" in data_rent['quarter'][i]:
        year_total[2] += data_rent['median_rent'][i]
        count_year[2] += 1
    elif "2008" in data_rent['quarter'][i]:
        year_total[3] += data_rent['median_rent'][i]
        count_year[3] += 1
    elif "2009" in data_rent['quarter'][i]:
        year_total[4] += data_rent['median_rent'][i]
        count_year[4] += 1
    elif "2010" in data_rent['quarter'][i]:
        year_total[5] += data_rent['median_rent'][i]
        count_year[5] += 1
    elif "2011" in data_rent['quarter'][i]:
        year_total[6] += data_rent['median_rent'][i]
        count_year[6] += 1
    elif "2012" in data_rent['quarter'][i]:
        year_total[7] += data_rent['median_rent'][i]
        count_year[7] += 1
    elif "2013" in data_rent['quarter'][i]:
        year_total[8] += data_rent['median_rent'][i]
        count_year[8] += 1
    elif "2014" in data_rent['quarter'][i]:
        year_total[9] += data_rent['median_rent'][i]
        count_year[9] += 1
    elif "2015" in data_rent['quarter'][i]:
        year_total[10] += data_rent['median_rent'][i]
        count_year[10] += 1
    elif "2016" in data_rent['quarter'][i]:
        year_total[11] += data_rent['median_rent'][i]
        count_year[11] += 1
    elif "2017" in data_rent['quarter'][i]:
        year_total[12] += data_rent['median_rent'][i]
        count_year[12] += 1
    elif "2018" in data_rent['quarter'][i]:
        year_total[13] += data_rent['median_rent'][i]
        count_year[13] += 1
    elif "2019" in data_rent['quarter'][i]:
        year_total[14] += data_rent['median_rent'][i]
        count_year[14] += 1
    elif "2020" in data_rent['quarter'][i]:
        year_total[15] += data_rent['median_rent'][i]
        count_year[15] += 1
    elif "2021" in data_rent['quarter'][i]:
        year_total[16] += data_rent['median_rent'][i]
        count_year[16] += 1
    elif "2022" in data_rent['quarter'][i]:
        year_total[17] += data_rent['median_rent'][i]
        count_year[17] += 1

for x in range(len(count_year)):
    avg_rent[x] = year_total[x]/count_year[x]



# Cleaning Data set 3

### For Scatter Plot:
| **Probem** | **Solution** |
| ----------- | ----------- |
| Amenities are split into 4 caterogies, need to calculate average number per town | counts the number of amenities per block and calculates the average number of amenities per town  |
| The town names in this data set are in abreviations while the other data sets have town names in full | Converts the abreviated town names to full using a dictionary |

### For Pie Chart:
| **Problem** | **Solution** |
| ----------- | ----------- |
| Need to find the average number of each type of flat | calculates the average number of each type of flat |
| The 4 room and 5 room rented flats are combined under 'other_room_rental' | Used the ratio of number of 4 and 5 room flats sold to extrapolate the number of 4 and 5 rooms rented (Explained further in Analysis of Pie Chart) |






In [ ]:
# Dictionary for town for data_info
town = {
    "AMK" : "Ang Mo Kio",
    "BB" : "Bukit Batok",
    "BD" : "Bedok",
    "BH" : "Bishan",
    "BM" : "Bukit Merah",
    "BP" : "Bukit Panjang",
    "BT" : "Bukit Timah",
    "CCK" : "Choa Chu Kang",
    "CL" : "Clementi",
    "CT" : "Central Area",
    "GL" : "Geylang",
    "HG" : "Hougang",
    "JE" : "Jurong East",
    "JW" : "Jurong West",
    "KWN" : "Kallang/Whampoa",
    "MP" : "Marine Parade",
    "PG" : "Punggol",
    "PRC" : "Pasir Ris",
    "QT" : "Queenstown",
    "SB" : "Sembawang",
    "SGN" : "Serangoon",
    "SK" : "Sengkang",
    "TAP" : "Tampines",
    "TG" : "Tengah",
    "TP" : "Toa Payoh",
    "WL" : "Woodlands",
    "YS" : "Yishun", 
}

# Creating a new list to store the new data (amenities count)
amenities = []

# Create new list for the new town names
info_town_names = []

# Creates a new dictionary
average_amen = {}

# Creates a new list to calculate average amenities per town
avg = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

# Create a new list to count the number of each type of flats [2-room, 3-room, 4 and 5 room]
type_count = [0,0,0,0]

# Find the ratio of 4 room and 5 room so extrapolate the rental number as 4 and 5 room rental count is combined in the data
total_45 = sum(data_info["4room_sold"]) + sum(data_info["5room_sold"])
ratio4room = sum(data_info["4room_sold"])/total_45
ratio5room = sum(data_info["5room_sold"])/total_45

# Iterating through data_info
for i in range(len(data_info)):
    
# Counting the number of amenities for different blocks and stores data into the newly created list 
    x = 0
    if data_info["market_hawker"][i] == "Y":
        x += 1
    if data_info["miscellaneous"][i] == "Y":
        x += 1
    if data_info["multistorey_carpark"][i] == "Y":
        x += 1
    if data_info["precinct_pavilion"][i] == "Y":
        x += 1
    amenities.append(x)

# Convert town names from short form to full form and stores new names into new lis
    info_town_names.append(town[data_info['bldg_contract_town'][i]].upper())

# Calculate the total number of amenities in each town
    if data_info['bldg_contract_town'][i] == "AMK":
        avg[0] += x
        count[0] += 1
    elif data_info['bldg_contract_town'][i] == "BB":
        avg[1] += x
        count[1] += 1
    elif data_info['bldg_contract_town'][i] == "BD":
        avg[2] += x
        count[2] += 1
    elif data_info['bldg_contract_town'][i] == "BH":
        avg[3] += x
        count[3] += 1
    elif data_info['bldg_contract_town'][i] == "BM":
        avg[4] += x
        count[4] += 1
    elif data_info['bldg_contract_town'][i] == "BP":
        avg[5] += x
        count[5] += 1
    elif data_info['bldg_contract_town'][i] == "BT":
        avg[6] += x
        count[6] += 1
    elif data_info['bldg_contract_town'][i] == "CCK":
        avg[7] += x
        count[7] += 1
    elif data_info['bldg_contract_town'][i] == "CL":
        avg[8] += x
        count[8] += 1
    elif data_info['bldg_contract_town'][i] == "CT":
        avg[9] += x
        count[9] += 1
    elif data_info['bldg_contract_town'][i] == "GL":
        avg[10] += x
        count[10] += 1
    elif data_info['bldg_contract_town'][i] == "HG":
        avg[11] += x
        count[11] += 1
    elif data_info['bldg_contract_town'][i] == "JE":
        avg[12] += x
        count[12] += 1
    elif data_info['bldg_contract_town'][i] == "JW":
        avg[13] += x
        count[13] += 1
    elif data_info['bldg_contract_town'][i] == "KWN":
        avg[14] += x
        count[14] += 1
    elif data_info['bldg_contract_town'][i] == "MP":
        avg[15] += x
        count[15] += 1
    elif data_info['bldg_contract_town'][i] == "PG":
        avg[16] += x
        count[16] += 1
    elif data_info['bldg_contract_town'][i] == "PRC":
        avg[17] += x
        count[17] += 1
    elif data_info['bldg_contract_town'][i] == "QT":
        avg[18] += x
        count[18] += 1
    elif data_info['bldg_contract_town'][i] == "SB":
        avg[19] += x
        count[19] += 1
    elif data_info['bldg_contract_town'][i] == "SGN":
        avg[20] += x
        count[20] += 1
    elif data_info['bldg_contract_town'][i] == "SK":
        avg[21] += x
        count[21] += 1
    elif data_info['bldg_contract_town'][i] == "TAP":
        avg[22] += x
        count[22] += 1
    elif data_info['bldg_contract_town'][i] == "TG":
        avg[23] += x
        count[23] += 1
    elif data_info['bldg_contract_town'][i] == "TP":
        avg[24] += x
        count[24] += 1
    elif data_info['bldg_contract_town'][i] == "WL":
        avg[25] += x
        count[25] += 1
    elif data_info['bldg_contract_town'][i] == "YS":
        avg[26] += x
        count[26] += 1

# counts the number of each type of flats
    type_count[0] += data_info['2room_sold'][i] + data_info['2room_rental'][i]
    type_count[1] += data_info['3room_sold'][i] + data_info['3room_rental'][i]
    type_count[2] += data_info['4room_sold'][i] + int(data_info['other_room_rental'][i] * ratio4room)
    type_count[3] += data_info['5room_sold'][i] + int(data_info['other_room_rental'][i] * ratio5room)

# Calculates the average number and stores it in the dictionary (average_amen)
average_amen["ANG MO KIO"] = round(avg[0]/count[0],2)
average_amen["BUKIT BATOK"] = round(avg[1]/count[1],2)
average_amen["BEDOK"] = round(avg[2]/count[2],2)
average_amen["BISHAN"] = round(avg[3]/count[3],2)
average_amen["BUKIT MERAH"] = round(avg[4]/count[4],2)
average_amen["BUKIT PANJANG"] = round(avg[5]/count[5],2)
average_amen["BUKIT TIMAH"] = round(avg[6]/count[6],2)
average_amen["CHOA CHU KANG"] = round(avg[7]/count[7],2)
average_amen["CLEMENTI"] = round(avg[8]/count[8],2)
average_amen["CENTRAL AREA"] = round(avg[9]/count[9],2)
average_amen["GEYLANG"] = round(avg[10]/count[10],2)
average_amen["HOUGANG"] = round(avg[11]/count[11],2)
average_amen["JURONG EAST"] = round(avg[12]/count[12],2)
average_amen["JURONG WEST"] = round(avg[13]/count[13],2)
average_amen["KALLANG/WHAMPOA"] = round(avg[14]/count[14],2)
average_amen["MARINE PARADE"] = round(avg[15]/count[15],2)
average_amen["PUNGGOL"] = round(avg[16]/count[16],2)
average_amen["PASIR RIS"] = round(avg[17]/count[17],2)
average_amen["QUEENSTOWN"] = round(avg[18]/count[18],2)
average_amen["SEMBAWANG"] = round(avg[19]/count[19],2)
average_amen["SERANGOON"] = round(avg[20]/count[20],2)
average_amen["SENGKANG"] = round(avg[21]/count[21],2)
average_amen["TAMPINES"] = round(avg[22]/count[22],2)
average_amen["TENGAH"] = 0
average_amen["TOA PAYOH"] = round(avg[24]/count[24],2)
average_amen["WOODLANDS"] = round(avg[25]/count[25],2)
average_amen["YISHUN"] = round(avg[26]/count[26],2)


print(f"There are {amenities.count(0)} blocks with 0 amenities. \n")
print(f"There are {amenities.count(1)} blocks with 1 amenities. \n")
print(f"There are {amenities.count(2)} blocks with 2 amenities. \n")
print(f"There are {amenities.count(3)} blocks with 3 amenities. \n")
print(f"There are {amenities.count(4)} blocks with 4 amenities. \n")


# Graph _:  Relationship between number of amenities and flat price. (Scatter Plot)

In [ ]:
# Get 'resale_price' values and stores as integer type array
price = data_sale["resale_price"]

#get the amenities value for the data
val = []
for t in data_sale['town']:
    val.append(average_amen[t])

# Change the style of the figure
plt.style.use('dark_background')
fig , ax3 = plt.subplots(figsize=(7,7))

# Creates the scatterplot
sc_plt = ax3.scatter(val,price,c=price,s=5)

# Sets the title, x-axis label and y-axis label
ax3.set_xlabel("Amenities value", weight = 'bold', size = 12)
ax3.set_ylabel("Resale Price($)", weight = 'bold', size = 12)
ax3.set_title("Amenities VS Resale Price", weight = 'bold', size = 14)

# Sets the Y tick labels to integer from 0 to 15000000 with increments of 200000
ax3.set_yticklabels(np.arange(0,1500000,200000))

#Colorbar
plt.colorbar(sc_plt, format = '$%d')


# Scatter plot Analysis:

This scatter plot shows the relationship between the number of amenities against its resale price.

Generally, the more amenities around the flat, the higher the resale price. This plot indicates a moderate, positive, linear relationship between the number of amenities around the flat and the resale price of the flats.

From this plot, we can conclude that the price of the flats increases in value if more amenities are built around it, as the amenities makes it more convenient for the residents there.

Buyers may opt for houses with less amenities for a lower price or go for houses with more amenities at a higher price.

# Graph _: Rental Price Trends (Line Graph)

In [ ]:
fig, line_plt = plt.subplots(figsize=(15,8))

#Plotting line
line_plt.plot(years,avg_rent, marker='*',markersize= 9)

# Creating the labels in the graphs
line_plt.set_ylabel("Average Median Rental Price ($)",weight='bold',size=15)
line_plt.set_xlabel("Year",weight='bold',size=15)
line_plt.set_title("Average median Rental Price by Year",weight='bold',size=15)

#Show legend
line_plt.legend(loc=2,prop={'size': 15})


# Analysis of Line Graph

The X-axis of the graph is the year from 2005 to 2022, and the Y-axis is the average median rental price of each year.

From the line graph, it can be seen that the average median price rental price of flats have been increasing from 2005 to 2022.





# Graph _: Proportion of flat types (Pie Chart)

In [ ]:
# Create the labels for each catergory of the pie chart
labels = ['2 room', '3 room', '4 room', '5 room']

# Extrude the portion with the highest ratio
extrude = [0,0,0,0]
extrude[type_count.index(max(type_count))] = 0.1

# Configuring the colors of the pie chart
color = ['#6495ED','#0096FF','#000080','#4682B4']

# Configure the size of the display
fig , pie_plt = plt.subplots(figsize=(7,7))

# Creating the title of the plot
pie_plt.set_title("Proportion of each type of flats")

# Plotting the Pie Chart
pie_plt.pie(type_count, explode=extrude, labels=labels, autopct='%1.1f%%', startangle=45, shadow=True, colors=color)

# changing pie chart to donut chart
centre_circle = plt.Circle((0,0),0.77,fc='black')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
pie_plt.axis('equal')  
plt.tight_layout()
plt.show()

# Analysis of Pie Chart

Since the data had combined the number of 4 room rented and 5 room rented under 'other_room_rental', the data had to be extrapolated using the ratio of number of 5 rooms sold and the ratio of number of 4 room sold. This is done by calculating the sum of '4room_sold' and '5room_sold' and taking each value over the total. The ratio is then used to split the 'other_room_rental' to 4 room and 5 room. This is shown above in "Cleaning data set 3".

The pie chart shows there is the highest amount of 4 room flats followed by the 5 room and 3 room with about the same amount of flats. 

